In [19]:
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 300)
from pyDEA.core.data_processing.read_data_from_xls import construct_input_data_instance
from pyDEA.core.models.envelopment_model import EnvelopmentModelInputOriented,EnvelopmentModelOutputOriented
from pyDEA.core.models.envelopment_model_base import EnvelopmentModelBase
from pyDEA.core.models.bound_generators import generate_upper_bound_for_efficiency_score,generate_lower_bound_for_efficiency_score
from pyDEA.core.models.envelopment_model_decorators import DefaultConstraintCreator


from pyDEA.core.models.multiplier_model import MultiplierInputOrientedModel
from pyDEA.core.models.multiplier_model_base import MultiplierModelBase


In [20]:
dados = pd.read_csv('teste_dea.csv')
dmus = dados['placa'].values
coefs = dados.drop(['placa'], axis=1).values
dc = {dmu:coef for dmu,coef in zip(dmus, coefs)}
dt = construct_input_data_instance(['E1','S1','E2'], dc)
dt.add_output_category('S1')
dt.add_input_category('E2')
dt.add_input_category('E1')

In [21]:
mdl = MultiplierInputOrientedModel()
mdl1 = MultiplierModelBase(dt,1.0e-8,mdl)
sol = mdl1.run()

/Users/1moi6/GitHub/Python/tccs/tccs/lib/python3.11/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/1moi6/GitHub/Python/tccs/tccs/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/z6/n6t25pkx2nl6t75_c5d0rm0h0000gn/T/e55d3402c5b9495889254e85c2457ee4-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/z6/n6t25pkx2nl6t75_c5d0rm0h0000gn/T/e55d3402c5b9495889254e85c2457ee4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 331 COLUMNS
At line 1302 RHS
At line 1629 BOUNDS
At line 1633 ENDATA
Problem MODEL has 326 rows, 3 columns and 969 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 173 (-153) rows, 2 (-1) columns and 346 (-623) elements
0  Obj 1.675e-05 Dual inf 215.85535 (1)
3  Obj 0.59729186
Optimal - objective value 0.59729186
After Postsolve, objective 0.59729186, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 0.5972918563 - 3 iterations t

In [26]:
sol._print_for_one_dmu('dmu_814')

DMU: SDS6G09
code:  dmu_814
LP status: Optimal
Efficiency score: 1
Lambda variables: {'dmu_814': 1.0}
Input duals: {'E1': 0.0040805427, 'E2': 1e-08}
Output duals: {'S1': 0.00047801147}


In [ ]:
mdl = EnvelopmentModelOutputOriented(lower_bound_generator=generate_lower_bound_for_efficiency_score)
mdl1 = EnvelopmentModelBase(dt,mdl,DefaultConstraintCreator())
sol = mdl1.run()

In [ ]:
mdl = EnvelopmentModelInputOriented(upper_bound_generator=generate_upper_bound_for_efficiency_score)
mdl1 = EnvelopmentModelBase(dt,mdl,DefaultConstraintCreator())
sol = mdl1.run()



In [23]:
solucao = []
for dmu in dt.DMU_codes:
    try:
        aux = {'placa':dt.get_dmu_user_name(dmu),'dmu':dmu,'eficiencia':sol.get_efficiency_score(dmu)}
    except:
        aux = {'placa':dt.get_dmu_user_name(dmu),'dmu':dmu,'eficiencia':0}  
    # aux = {'placa':dt.get_dmu_user_name(dmu),'dmu':dmu,'eficiencia':sol.get_efficiency_score(dmu)}
    solucao.append(aux)

aux = pd.DataFrame(solucao)

xx = pd.merge(dados,aux, how='left', on='placa')


In [24]:
xx

,placa,I1,O1,I2,dmu,eficiencia
0,BDP4I30,296.10,909.0,1645.830063,dmu_1,0.381013
1,BDP8C60,289.63,1694.5,2402.741403,dmu_4,0.693865
2,BDP8C61,448.67,2414.0,2454.512637,dmu_7,0.668763
3,BDP8C62,313.10,1654.0,2862.513613,dmu_10,0.618818
4,BDP8C63,118.13,165.0,2690.218963,dmu_13,0.163617
...,...,...,...,...,...,...
320,SDS7D39,553.99,3225.0,1903.738483,dmu_961,0.749521
321,SDS7D40,324.79,1483.0,3178.551426,dmu_964,0.534870
322,SDS7D42,480.20,2350.0,1121.914762,dmu_967,0.642543
323,SDS7D45,213.42,1133.0,2768.211825,dmu_970,0.621873


In [ ]:
xx

In [ ]:
mdl.get_input_variable_coefficient(obj_variable='E1',input_category='')

In [ ]:
sol.add_lambda_variables()

In [ ]:
xx

In [ ]:
pd.merge(dados,xx, how='left', on='placa')